# Juice.jl: a library for advanced probabilistic inference

Juice is a Julia library enabling tractable advanced probabilistic inference by leveraging logical and probabilistic circuits!


Github: https://github.com/Juice-jl

Juice.jl is divided into two Julia modules. 
    - LogicCircuits 
    - ProbabilisticCircuits

#### Included in this demo:
    - Probabilistic circuits
       - Load already trained circuits from file
    - Few common (tractable) probabilistic inference queries
       - Complete Evidence (EVI)
       - Marginals (MAR)
       - Conditionals (CON)
    - Some advanced probabilistic reasoning: 
       - Expected Prediction

In [1]:
using CSV
using Statistics
using LogicCircuits
using ProbabilisticCircuits

### Insurance dataset
yearly health insurance costs of individuals living in the USA

#### Helper functions

In [2]:
# You can skip this part. Includes helper functions to make partial observations from arrays of strings
# so its easier to present.

# Make one observation from list of string describing the observation
#
# For example, ["smoker", "male"] sets 
#   1) The mentioned features to the correct values.
#   2) Every feature not mentioned to missing values. 
FEATURES = 36;
function make_one_observation(obs)
    result = Int8.(ones(FEATURES) * -1)
    for k in obs
        # Smoking
        if lowercase(k) == "smoker"
            result[7:8] .= [0, 1]
        elseif lowercase(k) == "!smoker"
            result[7:8] .= [1, 0]
        # Gender
        elseif lowercase(k) == "male"
            result[13:14] .= [1, 0]
        elseif lowercase(k) == "female"
            result[13:14] .= [0, 1]
        # Region
        elseif lowercase(k) == "southeast"
            result[9:12] .= [0, 0, 1, 0]
        elseif lowercase(k) == "southwest"
            result[9:12] .= [0, 1, 0, 0]
        # Child
        elseif lowercase(k) == "1-child"
            result[1:6] .= [0,1,0,0,0,0]
        end
    end
    result
end;

function make_observations(obs)
    count = size(obs)[1]
    result = Int8.(ones(count, 36) * -1)
    for i=1:count
        result[i, :] .= make_one_observation(obs[i])
    end
    XData(result)
end;

### load Insurance Dataset

In [3]:
train_x = Bool.(Matrix(CSV.read("insurance/insurance_train_x.csv")));
train_y = Matrix(CSV.read("insurance/insurance_train_y.csv"));

test_x = Bool.(Matrix(CSV.read("insurance/insurance_test_x.csv")));
test_y = Matrix(CSV.read("insurance/insurance_test_y.csv"));

valid_x = Bool.(Matrix(CSV.read("insurance/insurance_valid_x.csv")));
valid_y = Matrix(CSV.read("insurance/insurance_valid_y.csv"));

Here for the purpose of this demo, we load a pretrained probabilistic circuit:

In [4]:
pc = load_prob_circuit(zoo_psdd_file("insurance.psdd"));
println("Probablistic Circuit with $(size(pc)[1]) nodes")

Probablistic Circuit with 27493 nodes


### Structural Properties

By enforcing some properties on the structure of the circuits we can enable tractable operations.


Properties:
- Structured decomposable
- Smooth
- Deterministic

Check this tutorial on Probabilistic Circuits: http://web.cs.ucla.edu/~guyvdb/slides/AAAI20.pdf

### EVI: Complete Evidence Query

All features are observed, we want to compute the probability:

$$ P(x) $$

In [5]:
lls_train = log_proba(pc, XData(train_x));

In [6]:
println("size: $(length(lls_train))\nmean: $(mean(exp.(lls_train))) \n min: $(minimum(exp.(lls_train))) \n max: $(maximum(exp.(lls_train)))")

size: 935
mean: 0.00012709917450436578 
 min: 5.167066345129343e-7 
 max: 0.0012602689505798468


### MAR: Marginal Query (partial evidence)

Now, what happens if we only observe a subset of the features $X^o$? We want to compute:

$$ P(X^o) = \sum_{x^m} P(X^o X^m) $$

**Problem:** Computing above query is usually not tractable as it involves summing over exponential (infinite) possible worlds.

**Good News:** In probabilistic circuits, given some structural properties, we can do this tractably. No need to enumerate all possible worlds.

In [7]:
marg_data = make_observations( [["smoker"], 
                       ["female"], 
                       ["female", "smoker"], 
                       ["southeast", "male", "1-child", "smoker"]],
                    )
lls = log_proba(pc, marg_data);
exp.(lls)

4-element Array{Float64,1}:
 0.18403566812744448
 0.47178308014843423
 0.09623782748058202
 0.00096392685052339

#### Q: Probablity of being smoker?

In [8]:
marg_data = make_observations([["smoker"]])
lls = log_proba(pc, marg_data);
exp.(lls[1])

0.18403566812744448

#### Q: Probability of being female smoker?

In [9]:
marg_data = make_observations([["female", "smoker"]])
lls = log_proba(pc, marg_data);
exp.(lls[1])

0.09623782748058202

####  Q: Probability of being male smoker with one child living in the southeast?

In [10]:
marg_data = make_observations([["southeast", "male", "1-child", "smoker"]])
lls = log_proba(pc, marg_data);
exp.(lls[1])

0.00096392685052339

### CON: Conditional Queries

Given some observations $X^o$, we want to compute probabilities conditioned on the observations:

$$ P(Q \mid X^o) $$

if we can do marginals tractably, we can also do conditionals tractably:

$$ P(Q \mid X^o) = \cfrac{P(Q, X^o)}{P(X^o)} $$


####  Q: Probability of being smoker given the person is male?

In [11]:
marg_data = make_observations([["male", "smoker"],["male"]])
lls = exp.(log_proba(pc, marg_data));
pq = lls[1] / lls[2]
println(" P('smoker' | 'male') = $(pq) ")

 P('smoker' | 'male') = 0.14445798812533656 


####  Q: Probability of being smoker given the person lives in southeast?

In [12]:
marg_data = make_observations([["southeast", "smoker"], ["southeast"]])
lls = exp.(log_proba(pc, marg_data));
pq = lls[1] / lls[2]
println(" P('smoker' | 'southeast') = $(pq) ")

 P('smoker' | 'southeast') = 0.2301714416720071 


# Exploratory predictive analysis

What about reasoning about predictive models such as regression models:


We are interested in computing **expected predictions**

- Appears all the time in machine learning, such as handling missing data
- We can do this tractably!


$$ \Large \mathbb{E}_{\mathbf{x}^m\ \sim\ p(\mathbf{x}^m\ \mid\ \mathbf{x}^o )}\left[\ f( \mathbf{x}^o \mathbf{x}^m) \ \right] $$

- In above equation $ \mathbf{x}^m $ = missing features, and $ \mathbf{x}^o $ = observed features.

- We have two separate models $p$ and $f$.

- Expected Prediction useful for:
  - Handling missing values at test time
  - Reasoning about behaviour of predictive models

Check out our paper: On Tractable Computation of Expected Predictions (NeurIPS 2019, https://arxiv.org/abs/1910.02182)

In [13]:
# Load Regression Circuit
rc = load_logistic_circuit(zoo_lc_file("insurance.circuit"), 1);

In [14]:
println("Regression Circuit with $(size(rc)[1]) nodes")

Regression Circuit with 1148 nodes


## Sample Queries

### Q1: How different are the insurance costs between smokers and non smokers?

In [15]:
data = make_observations([["!smoker"], 
                 ["smoker"]])
exps, exp_cache = Expectation(pc, rc, data);

In [16]:
println("Smoker    : \$ $(exps[2])");
println("Non-Smoker: \$ $(exps[1])");
println("Difference: \$ $(exps[2] - exps[1])");

Smoker    : $ 31355.32630488978
Non-Smoker: $ 8741.747258310648
Difference: $ 22613.57904657913


### Q2: Is the predictive model biased by gender?

In [17]:
data = make_observations([["male"],
                 ["female"]])
exps, exp_cache = Expectation(pc, rc, data);

In [18]:
println("Female  : \$ $(exps[2])");
println("Male    : \$ $(exps[1])");
println("Diff    : \$ $(exps[2] - exps[1])");

Female  : $ 14170.125469335406
Male    : $ 13196.548926381849
Diff    : $ 973.5765429535568


### Q3:  Expecation and standard devation of few subpopulations

In [19]:
data = make_observations( [["southeast", "male", "1-child", "smoker"], 
                 ["southwest", "male", "1-child", "smoker"]])
exps, exp_cache = Expectation(pc, rc, data);

In [20]:
# Computes the second moment
mom2, mom_cache = Moment(pc, rc, data, 2);

### Computing Standard Deviation

In [21]:
stds = sqrt.( mom2 - exps.^2 );

#### Living in South East, Smoker, Male, One child

In [22]:
println("mu: $(round(exps[1])), std = $(round(stds[1]))")

mu: 30975.0, std = 11229.0


#### Living in South West, Smoker, Male, One child

In [23]:
println("mu: $(round(exps[2])), std = $(round(stds[2]))")

mu: 27251.0, std = 7717.0
